# תרגיל PANDAS

במחברת זו הכנו עבורכם תרגילון קצר על PANDAS שיעזור לכם להטמיע שימוש בסיסי בספרייה ולוודא שאתם מספיק בקיאים בה לקראת הקורס עצמו

כל השאלות מתייחסות לקובץ pandas_ex_data.csv המצורף עם מחברת זו.

##  שאלה 1

טענו את הקובץ לטבלה. <br> 
כמה עמודות יש לטבלה? מה שמות העמודות? <br>
כמה שורות יש לטבלה? <br>
מאיזו סוג כל עמודה? 

In [11]:
import os
import pandas as pd

path = os.getcwd() + '\\pandas_ex_data.csv'
df = pd.read_csv(path)
print("num of columns: {0}".format(len(df.columns)))
print("columns names: {0}".format(df.columns.values))
print("num of rows: {0}".format(len(df.index)))
print("columns types: {0}".format(df.dtypes))

c:\dev\python-tutorial\pandas_ex_data.csv
num of columns: 4
columns names: ['id' 'teacher_id' 'father_id' 'birth_date']
num of rows: 534000
columns types: id              int64
teacher_id    float64
father_id     float64
birth_date     object
dtype: object


## שאלה 2

הוסיפו לטבלה עמודה מחושבת ששומרת את הגיל (נכון להיום) בשנים עבור כל אחד מהאנשים המופיעים בטבלה. <br>
ודאו נכונות תשובתכם. <br>
שימרו את הטבלה עם העמודה החדשה לקובץ CSV חדש עם שם משמעותי

In [34]:
import os
import pandas as pd
from datetime import datetime, timedelta 

path = os.getcwd() + '\\pandas_ex_data.csv'
outPath = os.getcwd() + '\\pandas_ex_data_with_age.csv'

df = pd.read_csv(path)
currentTime = datetime.now()

def ageInYears(birth_date, current_date):
    format_str = "%Y-%m-%d"
    birth_date_converted = datetime.strptime(birth_date, format_str)
    delta = current_date - birth_date_converted
    age = int(delta.days / 365.2425)     
    return age

df['age'] = [ageInYears(val, currentTime) for val in df['birth_date']]
df.to_csv(outPath)
print(df.head(5))


     id  teacher_id  father_id  birth_date  age
0  3000         NaN        NaN  1941-04-09   79
1  3001         NaN        NaN  1943-02-05   77
2  3002         NaN        NaN  1957-02-27   63
3  3003         NaN        NaN  1949-04-28   71
4  3004         NaN        NaN  1944-02-26   76


## שאלה 3

מצאו את כל האבות שהפרש הגילאים שלהם לאחד מהילדים שלהם הוא מעל 40 שנה. <br>
ודאו נכונות תשובתכם.

In [126]:
import os
import pandas as pd
from datetime import datetime, timedelta 

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("columns types: {0}".format(df.dtypes))
print("num of rows in original data: {0}".format(len(df.index)))
print(df.head(5))

fathers = df[['id', 'age']]
fathers.rename(columns={"age": "father_age"}, inplace=True)
fathers.rename(columns={"id": "father_id"}, inplace=True)
print("num of rows in fathers table: {0}".format(len(fathers.index)))
print(fathers.head(5))

childs = df.loc[df.father_id.notna(), ['id', 'father_id', 'age']]
childs['father_id'] = childs['father_id'].map(lambda x: int(x))
childs.rename(columns={"age": "child_age"}, inplace=True)
childs.rename(columns={"id": "child_id"}, inplace=True)
print("num of rows in childs table: {0}".format(len(childs.index)))
print(childs.head(5))

merged = pd.merge(childs, fathers, how='left', on='father_id')
print("num of rows in merged table: {0}".format(len(merged.index)))
print(merged.head(5))

merged = merged[merged['father_age'] - merged['child_age'] > 40]
print("num of rows in merged over 40: {0}".format(len(merged.index)))
print(merged.head(5))

unique_fathers = merged.father_id.unique()
merged = merged[merged['father_id'].isin(unique_fathers)]
print("num of rows in merged over 40 and unique fathers: {0}".format(len(merged.index)))
print(merged.head(5))

print("verification")
print("child with id 100001")
print(df.loc[df['id']==100001])
print("father with id 25521")
print(df.loc[df['id']==25521])
         

columns types: Unnamed: 0      int64
id              int64
teacher_id    float64
father_id     float64
birth_date     object
age             int64
dtype: object
num of rows in original data: 534000
   Unnamed: 0    id  teacher_id  father_id  birth_date  age
0           0  3000         NaN        NaN  1941-04-09   79
1           1  3001         NaN        NaN  1943-02-05   77
2           2  3002         NaN        NaN  1957-02-27   63
3           3  3003         NaN        NaN  1949-04-28   71
4           4  3004         NaN        NaN  1944-02-26   76
num of rows in fathers table: 534000
   father_id  father_age
0       3000          79
1       3001          77
2       3002          63
3       3003          71
4       3004          76
num of rows in childs table: 500000
       child_id  father_id  child_age
34000    100000      39203         12
34001    100001      25521         11
34002    100002      43095         16
34003    100003      33185         10
34004    100004      42904   

## שאלה 4

חשבו טבלה ששומרת עבור כל מורה כמה תלמידים הוא מלמד. <br>
מי המורה עם הכי הרבה תלמידים? כמה תלמידים יש לו? <br>
מי המורה עם הכי מעט תלמידים? כמה תלמידים יש לו? <br>

In [136]:
import os
import pandas as pd
from datetime import datetime, timedelta 

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("num of rows in original data: {0}".format(len(df.index)))
print(df.head(5))

group_by_teacher = df.groupby(by="teacher_id")
students_count_per_teacher = group_by_teacher.size().to_frame('size')
print(students_count_per_teacher)

print("teacher with max students")
teacher_with_max_students = students_count_per_teacher.loc[students_count_per_teacher['size'] == students_count_per_teacher['size'].max()]
print(teacher_with_max_students)

print("teacher with min students")
teacher_with_min_students = students_count_per_teacher.loc[students_count_per_teacher['size'] == students_count_per_teacher['size'].min()]
print(teacher_with_min_students)

num of rows in original data: 534000
   Unnamed: 0    id  teacher_id  father_id  birth_date  age
0           0  3000         NaN        NaN  1941-04-09   79
1           1  3001         NaN        NaN  1943-02-05   77
2           2  3002         NaN        NaN  1957-02-27   63
3           3  3003         NaN        NaN  1949-04-28   71
4           4  3004         NaN        NaN  1944-02-26   76
            size
teacher_id      
3000.0       145
3001.0       140
3002.0       133
3003.0       148
3004.0       156
...          ...
71995.0      129
71996.0      111
71997.0      133
71998.0      120
71999.0      126

[4000 rows x 1 columns]
teacher with max students
            size
teacher_id      
3977.0       183
teacher with min students
            size
teacher_id      
71409.0       84


## שאלה 5

מה מספר התלמידים הממוצע לכל מורה?


In [138]:
import os
import pandas as pd
from datetime import datetime, timedelta 

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("num of rows in original data: {0}".format(len(df.index)))
print(df.head(5))

group_by_teacher = df.groupby(by="teacher_id")
students_count_per_teacher = group_by_teacher.size().to_frame('size')
print(students_count_per_teacher)

average_students_per_teacher = students_count_per_teacher['size'].mean()
print("average students count per teacher")
print(average_students_per_teacher)

num of rows in original data: 534000
   Unnamed: 0    id  teacher_id  father_id  birth_date  age
0           0  3000         NaN        NaN  1941-04-09   79
1           1  3001         NaN        NaN  1943-02-05   77
2           2  3002         NaN        NaN  1957-02-27   63
3           3  3003         NaN        NaN  1949-04-28   71
4           4  3004         NaN        NaN  1944-02-26   76
            size
teacher_id      
3000.0       145
3001.0       140
3002.0       133
3003.0       148
3004.0       156
...          ...
71995.0      129
71996.0      111
71997.0      133
71998.0      120
71999.0      126

[4000 rows x 1 columns]
average students count per teacher
133.0


## שאלה 6

מה הפרש הגילאים הגדול ביותר בין 2 תלמידים של אותו המורה? <br>
ודאו נכונות תשובתכם

In [165]:
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 5)

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("num of rows in original data: {0}".format(len(df.index)))
print(df)

group_by_teacher_min_age = df.groupby(by="teacher_id")['age'].min().to_frame('minAge')
print(group_by_teacher_min_age)

group_by_teacher_max_age = df.groupby(by="teacher_id")['age'].max().to_frame('maxAge')
print(group_by_teacher_max_age)

group_by_teacher_merge_age = pd.merge(group_by_teacher_min_age, group_by_teacher_max_age, how='left', on='teacher_id')
print(group_by_teacher_merge_age)

group_by_teacher_merge_age['age_diff'] = group_by_teacher_merge_age['maxAge'] - group_by_teacher_merge_age['minAge']
print(group_by_teacher_merge_age)

teacher_with_max_students_age_diff = group_by_teacher_merge_age.loc[group_by_teacher_merge_age['age_diff']==group_by_teacher_merge_age['age_diff'].max()]
print(teacher_with_max_students_age_diff)



num of rows in original data: 534000
        Unnamed: 0      id  teacher_id  father_id  birth_date  age
0                0    3000         NaN        NaN  1941-04-09   79
1                1    3001         NaN        NaN  1943-02-05   77
...            ...     ...         ...        ...         ...  ...
533998      533998  599998     71964.0    34623.0  2009-04-19   11
533999      533999  599999      3534.0    34653.0  2005-06-23   15

[534000 rows x 6 columns]
            minAge
teacher_id        
3000.0           8
3001.0           8
...            ...
71998.0          8
71999.0          8

[4000 rows x 1 columns]
            maxAge
teacher_id        
3000.0          55
3001.0          53
...            ...
71998.0         17
71999.0         17

[4000 rows x 1 columns]
            minAge  maxAge
teacher_id                
3000.0           8      55
3001.0           8      53
...            ...     ...
71998.0          8      17
71999.0          8      17

[4000 rows x 2 columns]
    

## שאלה 7

מי המורה הצעיר ביותר? מה גילו?

In [177]:
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 5
pd.set_option("display.max_rows", 5)

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("num of rows in original data: {0}".format(len(df.index)))
print(df)

teachers = df.loc[df.teacher_id.notna(), ['teacher_id']]
teachers['teacher_id'] = teachers['teacher_id'].map(lambda x: int(x))
print("all teachers")
print(teachers)

unique_teachers = teachers['teacher_id'].unique()
print("all unique teachers length: {}".format(len(unique_teachers)))
print(unique_teachers)

all_teachers_age = df.loc[df['id'].isin(unique_teachers)]
print("all teachers with age")
print(all_teachers_age)

youngest_teachers = all_teachers_age.loc[all_teachers_age['age'] == all_teachers_age['age'].min()]
print("youngest teachers")
print(youngest_teachers)



num of rows in original data: 534000
        Unnamed: 0      id  teacher_id  father_id  birth_date  age
0                0    3000         NaN        NaN  1941-04-09   79
1                1    3001         NaN        NaN  1943-02-05   77
...            ...     ...         ...        ...         ...  ...
533998      533998  599998     71964.0    34623.0  2009-04-19   11
533999      533999  599999      3534.0    34653.0  2005-06-23   15

[534000 rows x 6 columns]
all teachers
        teacher_id
2000          3737
2001          3886
...            ...
533998       71964
533999        3534

[532000 rows x 1 columns]
all unique teachers length: 4000
[ 3737  3886  4572 ... 71635 70557 70179]
all teachers with age
      Unnamed: 0     id  teacher_id  father_id  birth_date  age
0              0   3000         NaN        NaN  1941-04-09   79
1              1   3001         NaN        NaN  1943-02-05   77
...          ...    ...         ...        ...         ...  ...
3998        3998  71998    

## שאלה 8

מצאו תלמיד ומורה שהפרש הגילאים ביניהם הוא הקטן ביותר. מה הפרש זה ומי התלמיד והמורה?

In [185]:
import os
import pandas as pd
from datetime import datetime, timedelta 

path = os.getcwd() + '\\pandas_ex_data_with_age.csv'
df = pd.read_csv(path)
print("columns types: {0}".format(df.dtypes))
print("num of rows in original data: {0}".format(len(df.index)))
print(df.head(5))

teachers = df[['id', 'age']]
teachers.rename(columns={"age": "teacher_age"}, inplace=True)
teachers.rename(columns={"id": "teacher_id"}, inplace=True)
print("num of rows in teachers table: {0}".format(len(teachers.index)))
print(teachers.head(5))

students = df.loc[df.teacher_id.notna(), ['id', 'teacher_id', 'age']]
students['teacher_id'] = students['teacher_id'].map(lambda x: int(x))
students.rename(columns={"age": "student_age"}, inplace=True)
students.rename(columns={"id": "student_id"}, inplace=True)
print("num of rows in students table: {0}".format(len(students.index)))
print(students.head(5))

merged = pd.merge(students, teachers, how='left', on='teacher_id')
print("num of rows in merged table: {0}".format(len(merged.index)))
print(merged.head(5))

merged['age_diff'] = merged['teacher_age'] - merged['student_age']
print("num of rows in merged with age_diff: {0}".format(len(merged.index)))
print(merged.head(5))

min_diff_between_teacher_and_student = merged.loc[merged['age_diff']==merged['age_diff'].min()]
print("min age diff between teacher and student")
print(min_diff_between_teacher_and_student)


columns types: Unnamed: 0     int64
id             int64
               ...  
birth_date    object
age            int64
Length: 6, dtype: object
num of rows in original data: 534000
   Unnamed: 0    id  teacher_id  father_id  birth_date  age
0           0  3000         NaN        NaN  1941-04-09   79
1           1  3001         NaN        NaN  1943-02-05   77
2           2  3002         NaN        NaN  1957-02-27   63
3           3  3003         NaN        NaN  1949-04-28   71
4           4  3004         NaN        NaN  1944-02-26   76
num of rows in teachers table: 534000
   teacher_id  teacher_age
0        3000           79
1        3001           77
2        3002           63
3        3003           71
4        3004           76
num of rows in students table: 532000
      student_id  teacher_id  student_age
2000       70000        3737           27
2001       70001        3886           40
2002       70002        4572           30
2003       70003        3329           29
2004      

## שאלה 9

נסחו עוד שאלה מעניינת בנוגע לקובץ זה וענו עליה